In [7]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# =============================================
# 🧪 第三步：模型评估 & Ensemble 对比（test set）
# =============================================
# 👉 test_models.py
# 将读取3个模型，分别预测 test set，输出 IoU / F1 / Accuracy，并执行 voting ensemble。

# 💡 整个 pipeline 会在你完成图像上传后全部串起来。
# =============================================
# 🧪 第五步：模型评估 & Ensemble 对比（test_models.py）
# =============================================
import numpy as np
import os
import tensorflow as tf
import cv2
import glob
from tqdm import tqdm
from sklearn.metrics import (
    f1_score, jaccard_score, accuracy_score,
    precision_score, recall_score, confusion_matrix
)

# 模型路径
unet_path = '/content/drive/My Drive/unet_model.h5'
segnet_path = '/content/drive/My Drive/segnet_model.h5'
fcn_path = '/content/drive/My Drive/fcn_model.h5'

# 加载模型
unet = tf.keras.models.load_model(unet_path)
segnet = tf.keras.models.load_model(segnet_path)
fcn = tf.keras.models.load_model(fcn_path)

# 加载 test 数据
test_img_path = '/content/drive/My Drive/dataset/test/images/'
test_mask_path = '/content/drive/My Drive/dataset/test/masks/'
img_size = (256, 256)

def load_data(img_dir, mask_dir):
    images, masks = [], []
    files = sorted(glob.glob(img_dir + '*'))
    for path in tqdm(files):
        fname = os.path.basename(path)
        mask_path = mask_dir + fname.replace('.jpg', '.png')

        img = cv2.imread(path)
        img = cv2.resize(img, img_size)
        img = img / 255.0

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, img_size)
        mask = (mask > 127).astype(np.uint8)

        images.append(img)
        masks.append(mask)

    return np.array(images), np.array(masks)

x_test, y_test = load_data(test_img_path, test_mask_path)

# 预测
pred_unet = unet.predict(x_test) > 0.5
pred_segnet = segnet.predict(x_test) > 0.5
pred_fcn = fcn.predict(x_test) > 0.5

# Ensemble 投票（平均加权）
ensemble_pred = ((0.4 * pred_unet) + (0.3 * pred_segnet) + (0.3 * pred_fcn)) > 0.5

# 指标计算函数
def evaluate_model(preds, gts):
    f1, iou, acc, prec, rec = [], [], [], [], []
    for i in range(len(preds)):
        y_pred = preds[i].flatten().astype(np.uint8)
        y_true = gts[i].flatten().astype(np.uint8)

        f1.append(f1_score(y_true, y_pred, zero_division=1))
        iou.append(jaccard_score(y_true, y_pred, zero_division=1))
        acc.append(accuracy_score(y_true, y_pred))
        prec.append(precision_score(y_true, y_pred, zero_division=1))
        rec.append(recall_score(y_true, y_pred, zero_division=1))

    return {
        "F1": np.mean(f1),
        "IoU": np.mean(iou),
        "Accuracy": np.mean(acc),
        "Precision": np.mean(prec),
        "Recall": np.mean(rec),
    }

# 输出对比结果
def print_result(name, pred):
    metrics = evaluate_model(pred, y_test)
    print(f"{name}  →  F1: {metrics['F1']:.4f}  |  IoU: {metrics['IoU']:.4f}  |  Acc: {metrics['Accuracy']:.4f} | Prec: {metrics['Precision']:.4f} | Rec: {metrics['Recall']:.4f}")


print_result("U-Net", pred_unet)
print_result("SegNet", pred_segnet)
print_result("FCN", pred_fcn)
print_result("Ensemble", ensemble_pred)


100%|██████████| 1136/1136 [02:47<00:00,  6.76it/s]


36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step
U-Net  →  F1: 0.3953  |  IoU: 0.3209  |  Acc: 0.8579 | Prec: 0.7108 | Rec: 0.3767
SegNet  →  F1: 0.5348  |  IoU: 0.4437  |  Acc: 0.8423 | Prec: 0.6680 | Rec: 0.6114
FCN  →  F1: 0.3784  |  IoU: 0.2721  |  Acc: 0.8274 | Prec: 0.5569 | Rec: 0.3565
Ensemble  →  F1: 0.4690  |  IoU: 0.3822  |  Acc: 0.8706 | Prec: 0.7277 | Rec: 0.4479


In [6]:
# =============================================
# 🧪 第三步：模型评估 & Ensemble 对比（test set）
# =============================================
# 👉 test_models.py
# 将读取3个模型，分别预测 test set，输出 IoU / F1 / Accuracy，并执行 voting ensemble。

# 💡 整个 pipeline 会在你完成图像上传后全部串起来。
# =============================================
# 🧪 第五步：模型评估 & Ensemble 对比（test_models.py）
# =============================================
import numpy as np
import os
import tensorflow as tf
import cv2
import glob
from tqdm import tqdm
from sklearn.metrics import (
    f1_score, jaccard_score, accuracy_score,
    precision_score, recall_score, confusion_matrix
)

# 模型路径
unet_path = '/content/drive/My Drive/unet_model.h5'
segnet_path = '/content/drive/My Drive/segnet_model.h5'
fcn_path = '/content/drive/My Drive/fcn_model.h5'

# 加载模型
unet = tf.keras.models.load_model(unet_path)
segnet = tf.keras.models.load_model(segnet_path)
fcn = tf.keras.models.load_model(fcn_path)

# 加载 test 数据
test_img_path = '/content/drive/My Drive/dataset/test/images/'
test_mask_path = '/content/drive/My Drive/dataset/test/masks/'
img_size = (256, 256)

def load_data(img_dir, mask_dir):
    images, masks = [], []
    files = sorted(glob.glob(img_dir + '*'))
    for path in tqdm(files):
        fname = os.path.basename(path)
        mask_path = mask_dir + fname.replace('.jpg', '.png')

        img = cv2.imread(path)
        img = cv2.resize(img, img_size)
        img = img / 255.0

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, img_size)
        mask = (mask > 127).astype(np.uint8)

        images.append(img)
        masks.append(mask)

    return np.array(images), np.array(masks)

x_test, y_test = load_data(test_img_path, test_mask_path)

# 预测
pred_unet = unet.predict(x_test) > 0.5
pred_segnet = segnet.predict(x_test) > 0.5
pred_fcn = fcn.predict(x_test) > 0.5

# Ensemble 投票（平均加权）
ensemble_pred = (0.3 * pred_unet + 0.7 * pred_segnet) > 0.5
# 指标计算函数
def evaluate_model(preds, gts):
    f1, iou, acc, prec, rec = [], [], [], [], []
    for i in range(len(preds)):
        y_pred = preds[i].flatten().astype(np.uint8)
        y_true = gts[i].flatten().astype(np.uint8)

        f1.append(f1_score(y_true, y_pred, zero_division=1))
        iou.append(jaccard_score(y_true, y_pred, zero_division=1))
        acc.append(accuracy_score(y_true, y_pred))
        prec.append(precision_score(y_true, y_pred, zero_division=1))
        rec.append(recall_score(y_true, y_pred, zero_division=1))

    return {
        "F1": np.mean(f1),
        "IoU": np.mean(iou),
        "Accuracy": np.mean(acc),
        "Precision": np.mean(prec),
        "Recall": np.mean(rec),
    }

# 输出对比结果
def print_result(name, pred):
    metrics = evaluate_model(pred, y_test)
    print(f"{name}  →  F1: {metrics['F1']:.4f}  |  IoU: {metrics['IoU']:.4f}  |  Acc: {metrics['Accuracy']:.4f} | Prec: {metrics['Precision']:.4f} | Rec: {metrics['Recall']:.4f}")


print_result("U-Net", pred_unet)
print_result("SegNet", pred_segnet)
print_result("FCN", pred_fcn)
print_result("Ensemble", ensemble_pred)


100%|██████████| 1136/1136 [02:47<00:00,  6.79it/s]


36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step
U-Net  →  F1: 0.3953  |  IoU: 0.3209  |  Acc: 0.8579 | Prec: 0.7108 | Rec: 0.3767
SegNet  →  F1: 0.5348  |  IoU: 0.4437  |  Acc: 0.8423 | Prec: 0.6680 | Rec: 0.6114
FCN  →  F1: 0.3784  |  IoU: 0.2721  |  Acc: 0.8274 | Prec: 0.5569 | Rec: 0.3565
Ensemble  →  F1: 0.5348  |  IoU: 0.4437  |  Acc: 0.8423 | Prec: 0.6680 | Rec: 0.6114


In [10]:
import numpy as np
from sklearn.metrics import f1_score, jaccard_score, accuracy_score, precision_score, recall_score
import pandas as pd


# 假设你已经有这些模型的预测值和测试集
# pred_unet, pred_segnet, pred_fcn, y_test

# 统一维度
y_test_flat = y_test.reshape(len(y_test), -1)

def evaluate(pred_binary):
    f1, iou, acc, prec, rec = [], [], [], [], []
    for i in range(len(pred_binary)):
        y_pred_flat = pred_binary[i].flatten().astype(np.uint8)
        y_true_flat = y_test_flat[i]
        f1.append(f1_score(y_true_flat, y_pred_flat, zero_division=1))
        iou.append(jaccard_score(y_true_flat, y_pred_flat, zero_division=1))
        acc.append(accuracy_score(y_true_flat, y_pred_flat))
        prec.append(precision_score(y_true_flat, y_pred_flat, zero_division=1))
        rec.append(recall_score(y_true_flat, y_pred_flat, zero_division=1))
    return np.mean(f1), np.mean(iou), np.mean(acc), np.mean(prec), np.mean(rec)

# 多个融合策略
strategies = {
    "SegNet Only": (pred_segnet > 0.5),
    "0.5U + 0.5S": ((pred_unet + pred_segnet)/2 > 0.5),
    "0.2U + 0.8S": ((0.2*pred_unet + 0.8*pred_segnet) > 0.5),
    "0.6U + 0.4S": ((0.6*pred_unet + 0.4*pred_segnet) > 0.5),
    "0.2U + 0.6S + 0.2F": ((0.2*pred_unet + 0.6*pred_segnet + 0.2*pred_fcn) > 0.5),
    "Hard Voting (2/3)": (((pred_unet > 0.5).astype(int) + (pred_segnet > 0.5).astype(int) + (pred_fcn > 0.5).astype(int)) >= 2),
}

# 评估所有策略
results = []
for name, pred in strategies.items():
    f1, iou, acc, prec, rec = evaluate(pred)
    results.append([name, f1, iou, acc, prec, rec])

df_results = pd.DataFrame(results, columns=["Strategy", "F1", "IoU", "Accuracy", "Precision", "Recall"])
# 输出或保存结果
print("多融合策略性能对比：")
print(df_results)

# 保存为 CSV
df_results.to_csv("ensemble_results.csv", index=False)

多融合策略性能对比：
             Strategy        F1       IoU  Accuracy  Precision    Recall
0         SegNet Only  0.534768  0.443688  0.842313   0.667965  0.611405
1         0.5U + 0.5S  0.014965  0.014965  0.783105   1.000000  0.014965
2         0.2U + 0.8S  0.534768  0.443688  0.842313   0.667965  0.611405
3         0.6U + 0.4S  0.395311  0.320924  0.857895   0.710783  0.376717
4  0.2U + 0.6S + 0.2F  0.534768  0.443688  0.842313   0.667965  0.611405
5   Hard Voting (2/3)  0.469008  0.382171  0.870570   0.727656  0.447928


In [14]:
import numpy as np
from sklearn.metrics import f1_score, jaccard_score, accuracy_score, precision_score, recall_score
import pandas as pd

# 替换为你的预测结果和真实标签（必须是 numpy 数组，格式一致）
# pred_unet = unet.predict(x_test)
# pred_segnet = segnet.predict(x_test)
# pred_fcn = fcn.predict(x_test)
# y_test = ground truth, shape = (N, H, W)



# 评估函数
def evaluate(pred_binary):
    f1, iou, acc, prec, rec = [], [], [], [], []
    for i in range(len(pred_binary)):
        y_pred_flat = pred_binary[i].flatten().astype(np.uint8)
        y_true_flat = y_test_flat[i]
        f1.append(f1_score(y_true_flat, y_pred_flat, zero_division=1))
        iou.append(jaccard_score(y_true_flat, y_pred_flat, zero_division=1))
        acc.append(accuracy_score(y_true_flat, y_pred_flat))
        prec.append(precision_score(y_true_flat, y_pred_flat, zero_division=1))
        rec.append(recall_score(y_true_flat, y_pred_flat, zero_division=1))
    return np.mean(f1), np.mean(iou), np.mean(acc), np.mean(prec), np.mean(rec)

# 网格搜索最优加权组合（步长 0.1）
best_combo = ()
best_f1 = 0
results = []

step = 0.1
weights = np.arange(0.0, 1.0 + step, step)

for wu in weights:
    for ws in weights:
        if wu + ws > 1.0:
            continue
        wf = 1.0 - wu - ws
        ensemble = (wu * pred_unet + ws * pred_segnet + wf * pred_fcn) > 0.5
        f1, iou, acc, prec, rec = evaluate(ensemble)
        results.append([wu, ws, wf, f1, iou, acc, prec, rec])
        if f1 > best_f1:
            best_f1 = f1
            best_combo = (wu, ws, wf)

# 输出结果表格
df_results = pd.DataFrame(results, columns=["U-Net", "SegNet", "FCN", "F1", "IoU", "Accuracy", "Precision", "Recall"])
df_results_sorted = df_results.sort_values(by="F1", ascending=False).reset_index(drop=True)

print("多融合策略性能对比：")
print(df_results)

# 保存为 CSV
df_results.to_csv("ensemble_results.csv", index=False)

best_combo, best_f1


多融合策略性能对比：
    U-Net  SegNet           FCN        F1       IoU  Accuracy  Precision  \
0     0.0     0.0  1.000000e+00  0.499805  0.333171  0.500922   0.499835   
1     0.0     0.1  9.000000e-01  0.499536  0.332932  0.500702   0.499617   
2     0.0     0.2  8.000000e-01  0.499330  0.332748  0.500539   0.499454   
3     0.0     0.3  7.000000e-01  0.499510  0.332909  0.500574   0.499485   
4     0.0     0.4  6.000000e-01  0.498968  0.332426  0.500146   0.499060   
..    ...     ...           ...       ...       ...       ...        ...   
61    0.8     0.1  1.000000e-01  0.501224  0.334437  0.501377   0.500276   
62    0.8     0.2 -5.551115e-17  0.501028  0.334262  0.501174   0.500078   
63    0.9     0.0  1.000000e-01  0.500994  0.334233  0.501272   0.500175   
64    0.9     0.1 -2.775558e-17  0.500965  0.334208  0.501133   0.500029   
65    1.0     0.0  0.000000e+00  0.501189  0.334405  0.501337   0.500238   

      Recall  
0   0.499802  
1   0.499487  
2   0.499234  
3   0.499564  
4

((np.float64(0.8), np.float64(0.1), np.float64(0.09999999999999995)),
 np.float64(0.5012243860039799))

In [13]:
best_combo, best_f1

((np.float64(0.8), np.float64(0.1), np.float64(0.09999999999999995)),
 np.float64(0.5012243860039799))

In [15]:
import numpy as np
import tensorflow as tf
import cv2
import glob
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import (
    f1_score, jaccard_score,
    accuracy_score, precision_score,
    recall_score
)

# 模型加载——请确认路径正确
unet = tf.keras.models.load_model('/content/drive/My Drive/unet_model.h5')
segnet = tf.keras.models.load_model('/content/drive/My Drive/segnet_model.h5')
fcn = tf.keras.models.load_model('/content/drive/My Drive/fcn_model.h5')

# 测试集加载
def load_data(img_dir, mask_dir, img_size=(256,256)):
    images, masks = [], []
    filelist = sorted(glob.glob(img_dir + '*'))
    for p in tqdm(filelist):
        fname = p.split('/')[-1]
        img = cv2.imread(p)
        img = cv2.resize(img, img_size)/255.0
        mask = cv2.imread(mask_dir + fname.replace('.jpg','.png'), cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, img_size)
        mask = (mask > 127).astype(np.uint8)
        images.append(img); masks.append(mask)
    return np.array(images), np.array(masks)

x_test, y_test = load_data(
    '/content/drive/My Drive/dataset/test/images/',
    '/content/drive/My Drive/dataset/test/masks/'
)

# 模型预测（概率）
pred_unet = unet.predict(x_test)
pred_segnet = segnet.predict(x_test)
pred_fcn = fcn.predict(x_test)

# Flatten ground truth
y_test_flat = y_test.reshape(len(y_test), -1)

# 指标函数
def eval_metrics(pred_bin):
    arr = pred_bin.reshape(len(pred_bin), -1).astype(np.uint8)
    f1,iou,acc,prec,rec = [],[],[],[],[]
    for yt, yp in zip(y_test_flat, arr):
        f1.append(f1_score(yt, yp, zero_division=1))
        iou.append(jaccard_score(yt, yp, zero_division=1))
        acc.append(accuracy_score(yt, yp))
        prec.append(precision_score(yt, yp, zero_division=1))
        rec.append(recall_score(yt, yp, zero_division=1))
    return np.mean(f1), np.mean(iou), np.mean(acc), np.mean(prec), np.mean(rec)

# 网格搜索权重组合
results = []
best = None
best_f1 = -1

weights = np.arange(0.0, 1.0001, 0.1)

for wu in weights:
    for ws in weights:
        if wu + ws > 1.0: continue
        wf = 1.0 - wu - ws
        ensemble = (wu*pred_unet + ws*pred_segnet + wf*pred_fcn) > 0.5
        f1,iou,acc,prec,rec = eval_metrics(ensemble)
        results.append([wu, ws, wf, f1, iou, acc, prec, rec])
        if f1 > best_f1:
            best_f1 = f1
            best = (wu, ws, wf)

# 输出结果 DataFrame
df = pd.DataFrame(results,
    columns=["U-Net","SegNet","FCN","F1","IoU","Accuracy","Precision","Recall"])
df_sorted = df.sort_values(by="F1", ascending=False).reset_index(drop=True)

print("🏆 Best weights (by F1):", best, "→ F1:", best_f1)
print("\nTop combinations:")
print(df_sorted.head(10).to_string(index=False))

# 保存为 CSV
df_sorted.to_csv("grid_search_ensemble_results.csv", index=False)


100%|██████████| 1136/1136 [02:47<00:00,  6.79it/s]


36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step
🏆 Best weights (by F1): (np.float64(0.0), np.float64(1.0), np.float64(0.0)) → F1: 0.5347682647843448

Top combinations:
 U-Net  SegNet  FCN       F1      IoU  Accuracy  Precision   Recall
   0.0     1.0  0.0 0.534768 0.443688  0.842313   0.667965 0.611405
   0.0     0.9  0.1 0.534628 0.444496  0.847265   0.672848 0.602792
   0.0     0.8  0.2 0.534175 0.444805  0.853224   0.682341 0.591812
   0.1     0.9  0.0 0.531831 0.442478  0.846926   0.674956 0.599635
   0.0     0.7  0.3 0.531805 0.442877  0.860515   0.691548 0.576006
   0.1     0.8  0.1 0.531085 0.442885  0.852800   0.684556 0.588815
   0.1     0.7  0.2 0.530345 0.443047  0.860267   0.698698 0.575121
   0.2     0.8  0.0 0.527477 0.440308  0.852255   0.687251 0.584880
   0.1     0.6  0.3 0.526892 0.440091  0.869545   0.712703 0.552297
   0.2     0.7  0.1 0.526686 0.440799  0.859762   0.701266 0.571783
